In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation
from sklearn.ensemble import RandomForestRegressor

In [2]:
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']

In [3]:
data = yf.download(tickers, period="5y", interval="1d")["Adj Close"]

[*********************100%***********************]  15 of 15 completed


In [4]:
mu = expected_returns.mean_historical_return(data)
cov_matrix = risk_models.sample_cov(data)

In [5]:
def rf_objective(weights, rf_regressor, mu, cov_matrix, alpha=1):
    # Generate synthetic returns using Random Forest
    synthetic_returns = rf_regressor.predict(np.column_stack((mu, cov_matrix)).reshape(1, -1))

    # Calculate the portfolio expected return and volatility
    port_return = weights.dot(mu)
    port_volatility = np.sqrt(weights.dot(cov_matrix).dot(weights))

    # Maximize expected return and minimize volatility
    return -(port_return - alpha * port_volatility) / synthetic_returns

In [6]:
# Initialize the Random Forest regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

n_samples = len(mu)
rf_regressor.fit(mu.values.reshape(n_samples, 1), cov_matrix.values.reshape(n_samples, -1))

RandomForestRegressor(random_state=42)

In [7]:
ef = EfficientFrontier(mu, cov_matrix)
ef.add_objective(objective_functions.L2_reg)
ef.add_objective(lambda w: rf_objective(w, rf_regressor, mu, cov_matrix))
weights = ef.min_volatility()

ValueError: X has 240 features, but RandomForestRegressor is expecting 1 features as input.

In [ ]:
portfolio_value = 800000  # Set the portfolio value
da = DiscreteAllocation(weights, data.iloc[-1, :], total_portfolio_value=portfolio_value)
allocation, leftover = da.greedy_portfolio()

# Print the portfolio allocation and leftover amount
print("Portfolio allocation:")
for ticker, shares in allocation.items():
    print(f"{ticker}: {shares} shares")

print(f"\nLeftover: ${leftover:.2f}")